In [1]:
import io
from datetime import datetime, timedelta

import xarray as xr
import requests
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cmocean

import pandas as pd
import numpy as np

import tempfile
import wget

# Not used directly, but used via xarray
import cfgrib

In [2]:
metadata = pd.read_csv('../data/metadata.csv')

In [3]:
example_row = metadata[metadata.uid == "abau"].iloc[0]
example_row

uid                abau
latitude      35.682235
longitude    -79.207716
date         2016-06-21
split             train
Name: 38, dtype: object

In [4]:
# Constants for creating the full URL
blob_container = 'https://noaa-hrrr-bdp-pds.s3.amazonaws.com'
sector = "conus"
date = datetime.strptime(example_row.date, "%Y-%m-%d").date()
cycle = 1           # noon
forecast_hour = 1   # offset from cycle time
product = "wrfsfcf" # 2D surface levels

# Put it all together
file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
url = f"{blob_container}/hrrr.{date:%Y%m%d}/{sector}/{file_path}"

print(url)

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20160621/conus/hrrr.t01z.wrfsfcf01.grib2


In [5]:
date

datetime.date(2016, 6, 21)

In [6]:
file = tempfile.NamedTemporaryFile(prefix="tmp_",suffix=".txt")

file_name = wget.download(url, out=file.name)

100% [........................................................................] 82906618 / 82906618

In [7]:
file.name

'/tmp/tmp_65qqv0fr.txt'

In [10]:
ds = xr.open_dataset(file_name, engine='cfgrib', 
                     backend_kwargs={'indexpath':''},
                     filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})

In [26]:
lat = example_row.latitude
lon = example_row.longitude + 360

abslat = np.abs(ds.latitude-lat)
abslon = np.abs(ds.longitude-lon)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c))

meta_info = {'id': example_row['uid']}
for varname, da in ds.sel(y=xloc, x=yloc).data_vars.items():
    meta_info[da.attrs['long_name']] = da.values.item()


In [27]:
meta_info

{'id': 'abau',
 'Visibility': 17800.0,
 'Wind speed (gust)': 3.2077999114990234,
 'Lightning': 0.0,
 'Surface pressure': 100460.0,
 'Orography': 121.99525451660156,
 'Temperature': 295.07452392578125,
 'Water equivalent of accumulated snow depth (deprecated)': 0.0,
 'Snow cover': 0.0,
 'Snow depth': 0.0,
 'Percent frozen precipitation': -50.0,
 'Precipitation rate': 0.0,
 'Categorical snow': 0.0,
 'Categorical ice pellets': 0.0,
 'Categorical freezing rain': 0.0,
 'Categorical rain': 0.0,
 'Vegetation Type': 9.0,
 'Convective available potential energy': 0.0,
 'Convective inhibition': 0.0,
 'Downward short-wave radiation flux': 0.0,
 'Boundary layer height': 0.0,
 'Land-sea mask': 1.0,
 'Sea ice area fraction': 0.0}

In [ ]:
file.delete

In [13]:
file.close()

In [ ]:
file.delete

In [9]:
file_name

'/tmp/tmp_65qqv0fr (1).txt'

In [14]:
# Fetch the idx file by appending the .idx file extension to our already formatted URL
r = requests.get(f"{url}.idx")
idx = r.text.splitlines()

# Take a peek at the content of the index
print(*idx[0:10], sep="\n")

1:0:d=2016062101:REFC:entire atmosphere:1 hour fcst:
2:540770:d=2016062101:RETOP:cloud top:1 hour fcst:
3:827593:d=2016062101:VIL:entire atmosphere:1 hour fcst:
4:1234618:d=2016062101:VIS:surface:1 hour fcst:
5:2539732:d=2016062101:REFD:1000 m above ground:1 hour fcst:
6:2893979:d=2016062101:REFD:4000 m above ground:1 hour fcst:
7:3039499:d=2016062101:GUST:surface:1 hour fcst:
8:4437878:d=2016062101:HGT:500 mb:1 hour fcst:
9:5149039:d=2016062101:TMP:500 mb:1 hour fcst:
10:5712117:d=2016062101:DPT:500 mb:1 hour fcst:


In [15]:
len(idx)

102

In [16]:
print(*idx, sep="\n")

1:0:d=2016062101:REFC:entire atmosphere:1 hour fcst:
2:540770:d=2016062101:RETOP:cloud top:1 hour fcst:
3:827593:d=2016062101:VIL:entire atmosphere:1 hour fcst:
4:1234618:d=2016062101:VIS:surface:1 hour fcst:
5:2539732:d=2016062101:REFD:1000 m above ground:1 hour fcst:
6:2893979:d=2016062101:REFD:4000 m above ground:1 hour fcst:
7:3039499:d=2016062101:GUST:surface:1 hour fcst:
8:4437878:d=2016062101:HGT:500 mb:1 hour fcst:
9:5149039:d=2016062101:TMP:500 mb:1 hour fcst:
10:5712117:d=2016062101:DPT:500 mb:1 hour fcst:
11:6780489:d=2016062101:UGRD:500 mb:1 hour fcst:
12:7386056:d=2016062101:VGRD:500 mb:1 hour fcst:
13:7984286:d=2016062101:HGT:700 mb:1 hour fcst:
14:8689834:d=2016062101:TMP:700 mb:1 hour fcst:
15:9263249:d=2016062101:DPT:700 mb:1 hour fcst:
16:10476980:d=2016062101:UGRD:700 mb:1 hour fcst:
17:11089560:d=2016062101:VGRD:700 mb:1 hour fcst:
18:11703871:d=2016062101:HGT:850 mb:1 hour fcst:
19:12439452:d=2016062101:TMP:850 mb:1 hour fcst:
20:13024968:d=2016062101:DPT:850 mb:1 

In [29]:
ds.data_vars.items()

ItemsView(Data variables:
    vis      (y, x) float32 ...
    gust     (y, x) float32 ...
    ltng     (y, x) float32 ...
    sp       (y, x) float32 ...
    orog     (y, x) float32 ...
    t        (y, x) float32 ...
    sdwe     (y, x) float32 ...
    snowc    (y, x) float32 ...
    sde      (y, x) float32 ...
    cpofp    (y, x) float32 ...
    prate    (y, x) float32 ...
    csnow    (y, x) float32 ...
    cicep    (y, x) float32 ...
    cfrzr    (y, x) float32 ...
    crain    (y, x) float32 ...
    gppbfas  (y, x) float32 ...
    cape     (y, x) float32 ...
    cin      (y, x) float32 ...
    dswrf    (y, x) float32 ...
    blh      (y, x) float32 ...
    lsm      (y, x) float32 ...
    siconc   (y, x) float32 ...)

In [28]:
ds

<xarray.Dataset>
Dimensions:     (y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (y, x) float64 21.14 21.14 21.15 21.16 ... 47.86 47.85 47.84
    longitude   (y, x) float64 237.3 237.3 237.3 237.4 ... 299.0 299.0 299.1
    valid_time  datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables: (12/22)
    vis         (y, x) float32 ...
    gust        (y, x) float32 ...
    ltng        (y, x) float32 ...
    sp          (y, x) float32 ...
    orog        (y, x) float32 ...
    t           (y, x) float32 ...
    ...          ...
    cape        (y, x) float32 ...
    cin         (y, x) float32 ...
    dswrf       (y, x) float32 ...
    blh         (y, x) float32 ...
    lsm         (y, x) float32 ...
    siconc      (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2024-03-22T00:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [30]:
varname

'siconc'